In [85]:
import subprocess
import json
import os

from constants import *
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from eth_account import Account
from pathlib import Path
from getpass import getpass
from constants import *
load_dotenv()
from bit import *
from bit import Key
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

In [86]:
#Initiate Web3 object
web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
web3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [87]:
print(web3.isConnected())

True


In [88]:
print(web3.eth.blockNumber)

118


In [8]:
#Set as an Environment Variable at first
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

welcome speak kite beef best favorite initial assault tray enact occur report topic citizen ecology


In [9]:
def derive_wallets(mnem,coinname,num):
    command = './derive -g --mnemonic="'+str(mnem)+'" --numderive=""'+str(num)+'"" --coin=""'+str(coinname)+'"" --format=jsonpretty'
    p = subprocess.Popen(command,stdout=subprocess.PIPE,shell=True)
    (output, err) = p.communicate()
    return json.loads(output)

In [10]:

coins = {ETH:derive_wallets(mnem=mnemonic,coinname=ETH,num=3),BTCTEST: derive_wallets(mnem=mnemonic,coinname=BTCTEST,num=3)}

In [11]:
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xdf460bD851a7540c70BeE83E5bA1b63410c4F220',
   'xprv': 'xprvA464w7QMmMw46Qu6D8oqnXFqzN5h2zZi14TruAoAMas27BoHWiF5DtzV8E9jbJANQ9G7FZrLeHJV8P5Le52CgHGR86waA4b8AJkXamLiugj',
   'xpub': 'xpub6H5RLcwFbjVMJtyZKALr9fCaYPvBSTHZNHPThZCmuvPzyz8S4FZKmhJxyXdrvW6YmE7C7jztA4xXRLnAVk1HcqsMWuzAVc1zi9MMqnBF5Lw',
   'privkey': '0x179d74c8c264b83f5339a2c68b07136cf5501ed5b1391f9f869f8de68d8574a9',
   'pubkey': '035adc788a729e151491238e5cde3914c8f5e1a0d07fd3f6f960320e1609573270',
   'pubkeyhash': '7dcf5087fbb8fcb270f9a65670103bbca2cab9e7',
   'index': 0},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0xE43a083A830A8fFc70BeCC64e9A54369F83a3Ae8',
   'xprv': 'xprvA464w7QMmMw47wdFdjrGYSknKgvobkVTgCUGzYr4VUWkdcHS1oHpHhzQYaaJLoFp9FWzU3jXGw8XrK7jgPeCtiprfuiYnmoKe7pT4Tetk3f',
   'xpub': 'xpub6H5RLcwFbjVMLRhijmPGuahWsimJ1DDK3RPsnwFg3p3jWQcaZLc4qWJtProXuKWrbEJNZbsN4fcaMqLdYBfFDfiJuP5mXMFwZ3qXCxLfEPj',
   'privkey': '0xff6d5164d04dc008c1e6b3d7aa84dbfd5d22a983b0def6

In [150]:
def priv_key_to_account (coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [151]:
def create_raw_tx(account, recipient, amount):
    gasEstimate = web3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": web3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": web3.eth.getTransactionCount(account.address),
    }

In [152]:
def create_tx(coin,account,to,amount):
    if (coin==ETH):
        return create_raw_tx(account,to,amount)
    else:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [176]:
#This is usable for POW only not POA 
def send_tx (coin, account, recipient, amount):
    if coin =='ETH':
        trxns_eth = create_tx(coin,account, recipient, amount)
        sign_trxns_eth = account.sign_transaction(trxns_eth)
        result = w3.eth.sendRawTransaction(sign_trxns_eth.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trxns_btctest= create_tx(coin,account,recipient,amount)
        sign_trxns_btctest = account.sign_transaction(trxns_btctest)
        NetworkAPI.broadcast_tx_testnet(sign_trxns_btctest)       
        return tx_hex

In [163]:
eth_pk_acct1 = priv_key_to_account(ETH,coins['eth'][0]['privkey'])
eth_pk_acct2 = priv_key_to_account(ETH, coins['eth'][1]['privkey'])
eth_pk_acct3 = priv_key_to_account(ETH, coins['eth'][2]['privkey'])

In [164]:
eth_pk_acct2.address

'0xE43a083A830A8fFc70BeCC64e9A54369F83a3Ae8'

In [165]:
eth_pk_acct1.address

'0xdf460bD851a7540c70BeE83E5bA1b63410c4F220'

In [157]:
btc_pk_acct1 = priv_key_to_account('btc-test',coins['btc-test'][0]['privkey'])
btc_pk_acct2 = priv_key_to_account('btc-test',coins['btc-test'][1]['privkey'])

In [181]:
btc_pk_acct2.address

'my52hmV9JDd4aN6zvHaPDoU4L4RFj5sQ8K'

In [ ]:
send_tx('btc-test',btc_pk_acct1, btc_pk_acct2.address,.00005)

In [ ]:
# Here I was not able to transact 

send_tx('eth',eth_pk_acct1,'0xE43a083A830A8fFc70BeCC64e9A54369F83a3Ae8',.000002 )

# POA - follow steps bellow for POA only due to signers and needing to run the Bit-39 wallet via your own network. 

In [93]:
balance = web3.eth.getBalance("0xdf460bD851a7540c70BeE83E5bA1b63410c4F220")

In [94]:
balance

904625697166532776746648320380374280103671755200316906518261955061821325312

In [105]:
account1 = '0xdf460bD851a7540c70BeE83E5bA1b63410c4F220'

In [106]:
account2 = '0xE43a083A830A8fFc70BeCC64e9A54369F83a3Ae8'


In [116]:
account3 = '0xA99f8B8D0173eCcb91E57547f0Dd57D7c172D3F5'

In [110]:
privatekey = '0x179d74c8c264b83f5339a2c68b07136cf5501ed5b1391f9f869f8de68d8574a9'

In [60]:
send_tx('btc-test',btc_pk_acct1,btc_pk_acct2.address,0.0005)

In [107]:
nonce = web3.eth.getTransactionCount(account1)

tx = {
    'nonce': nonce,
    'to': account2,
    'value': web3.toWei(1, 'ether'),
    'gas': 2000000,
    'gasPrice': web3.toWei('50', 'gwei'),
}

In [111]:
signed_tx = web3.eth.account.signTransaction(tx, privatekey)

In [112]:
tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
print(web3.toHex(tx_hash))

0x32bc4449b1c6fcbde6e419603f3e67cb06faa1e1c0e439fbb39a69824e3ca7b2


In [120]:
nonce = web3.eth.getTransactionCount(account1)

tx = {
    'nonce': nonce,
    'to': account3,
    'value': web3.toWei(10, 'ether'),
    'gas': 2000000,
    'gasPrice': web3.toWei('50', 'gwei'),
}

In [121]:
signed_tx = web3.eth.account.signTransaction(tx, privatekey)

In [122]:
tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
print(web3.toHex(tx_hash))

0xe531132e7b5a22398cb362199d5963bf0f88b515da3782a9950cec1c909b40bd
